In [1]:
import pandas as pd

# index, session_id, song_id, unix_played_at, play_status, login_type, listening_order
test_source = pd.read_parquet("../../datagame-2023/label_test_source.parquet")
# index, song_id, artist_id, song_length, album_id, language_id, album_month
meta_song = pd.read_parquet("../../datagame-2023/meta_song.parquet")

In [2]:
test_session_to_songs = test_source.groupby('session_id')['song_id'].apply(list).to_dict()

In [3]:
from tqdm import tqdm

song_to_artist = dict()
song_to_album = dict()

for row in tqdm(meta_song.itertuples(), total=len(meta_song)):
    if not pd.isna(row.artist_id):
        song_to_artist[row.song_id] = int(row.artist_id)

    if not pd.isna(row.album_id):
        song_to_album[row.song_id] = int(row.album_id)

100%|██████████| 1030712/1030712 [00:01<00:00, 648575.29it/s]


In [4]:
del test_source, meta_song

In [5]:
from collections import Counter


class myConditionalFreqDist:
    def __init__(self):
        self._data = {}
        self.values = [1, 0.63, 0.5, 0.43, 0.38]  # ndcg 的加權分數

    def __getitem__(self, key):
        return self._data.get(key, {})

    def __setitem__(self, key, value):
        self._data[key] = value

    def inc(self, condition, sample, index):
        if condition in self._data:
            if sample in self._data[condition]:
                self._data[condition][sample] += self.values[index]
            else:
                self._data[condition][sample] = self.values[index]
        else:
            self._data[condition] = {sample: self.values[index]}

    def most_common(self, condition, n=None):
        if condition in self._data:
            counter = Counter(self._data[condition])
            return counter.most_common(n)
        else:
            return []

In [6]:
import pickle

In [7]:
with open('cfd_3grams_test', 'rb') as file:
    cfd_3grams = pickle.load(file)

In [8]:
with open('cfd_4grams_test', 'rb') as file:
    cfd_4grams = pickle.load(file)

In [9]:
with open('cfd_5grams_test', 'rb') as file:
    cfd_5grams = pickle.load(file)

In [10]:
with open('jmlm_0.9999_token10.pkl', 'rb') as file:
    results = pickle.load(file)

In [17]:
from tqdm import tqdm

submit_rows = []
results_index = 0
part_repeated = 0
part4321 = 0
part321 = 0
part21 = 0
part5432 = 0
part432 = 0
part6543 = 0
part543 = 0
part32 = 0
part321new1 = 0
part21new1 = 0
part21new1new2 = 0
part1new1new2 = 0
part_mle = 0

for key in tqdm(test_session_to_songs.keys(), total=len(test_session_to_songs.keys())):
    query_song = test_session_to_songs[key][:]

    '''
    載入當前session資訊
    '''
    session_id, song_ids = results[results_index]
    while session_id != key:
        results_index += 1
        session_id, song_ids = results[results_index]

    # 預測下一首歌曲
    predicted_songs = []

    '''
    用戶行為
    '''
    user_code = 0  # 0=正常用戶，1=單一藝人循環，2=單一專輯循環

    is_same_artist = True
    is_same_album = True
    artist = song_to_artist.get(query_song[-10], '')
    album = song_to_album.get(query_song[-10], '')
    for query in query_song[-10:]:
        if song_to_artist.get(query, ' ') != artist:
            is_same_artist = False
        if song_to_album.get(query, ' ') != album:
            is_same_album = False
    if is_same_artist:
        user_code = 1
    if is_same_album:
        user_code = 2

    if len(set(query_song[-2:])) == 1:  # 如果倒數兩首都是重複同一首歌
        predicted_songs.append(query_song[-1])
        part_repeated += 1

    '''
    5 4 3 grams 依次過濾，分數主要來源來自這三個，其他其實都只是加分項
    '''
                
    if len(predicted_songs) < 5:
        start_song = (query_song[-4] + query_song[-3] + query_song[-2] + query_song[-1])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part4321 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part4321 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs:
                part4321 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-3] + query_song[-2] + query_song[-1])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part321 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part321 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs:
                part321 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-2] + query_song[-1])
        next_songs = cfd_3grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part21 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part21 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part21 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    '''
    賭狗行為沒賭到，開始找如果不選 query_song[-1] 的選項
    依序是倒數
    5432
    432
    6543
    543
    32 的預測架構，盡量找長序列，然後加43雖然能找得到東西，但太遠了相關性就掉得厲害
    可能可以再改改順序，但算了不凹分
    '''
    if len(predicted_songs) < 5:
        start_song = (query_song[-5] + query_song[-4] + query_song[-3] + query_song[-2])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part5432 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part5432 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part5432 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-4] + query_song[-3] + query_song[-2])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part432 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part432 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part432 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-6] + query_song[-5] + query_song[-4] + query_song[-3])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part6543 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part6543 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part6543 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-5] + query_song[-4] + query_song[-3])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part543 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part543 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part543 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-3] + query_song[-2])
        next_songs = cfd_3grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]

            if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                songs = [song for song, song_times in next_songs if song_times == times]
                for song in song_ids:
                    if len(predicted_songs) >= 5:
                        break
                    if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                        part32 += 1
                        predicted_songs.append(song)
                for song in songs:
                    if len(predicted_songs) >= 5:
                        break
                    if (song not in predicted_songs) and (songs not in query_song):
                        part32 += 1
                        predicted_songs.append(song)
                index += len(songs)
                continue
            index += 1

            if next_song not in predicted_songs and (next_songs not in query_song):
                part32 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    '''
    賭狗上線
    預測順序是 5gram 4gram 先加入第一個變數來預測
    接下來是 5gram 4gram 加入第二個變數來預測，因為加入越後面的變數越容易偏掉（可能冷門歌曲亂跳)
    加入第三個之後模型預測效能會下降 (0.56056->0.56051)
    '''
    if 0 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            start_song = (query_song[-3] + query_song[-2] + query_song[-1] + predicted_songs[i])
            next_songs = cfd_5grams.most_common(start_song)

            index = 0
            while index < len(next_songs) and len(predicted_songs) < 5:
                next_song, times = next_songs[index]

                if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                    songs = [song for song, song_times in next_songs if song_times == times]
                    for song in song_ids:
                        if len(predicted_songs) >= 5:
                            break
                        if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                            part321new1 += 1
                            predicted_songs.append(song)
                    for song in songs:
                        if len(predicted_songs) >= 5:
                            break
                        if (song not in predicted_songs) and (songs not in query_song):
                            part321new1 += 1
                            predicted_songs.append(song)
                    index += len(songs)
                    continue
                index += 1

                if next_song not in predicted_songs and (next_songs not in query_song):
                    part321new1 += 1
                    predicted_songs.append(next_song)
                if len(predicted_songs) >= 5:
                    break

            if len(predicted_songs) >= 5:
                break

    if 0 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            start_song = (query_song[-2] + query_song[-1] + predicted_songs[i])
            next_songs = cfd_4grams.most_common(start_song)

            index = 0
            while index < len(next_songs) and len(predicted_songs) < 5:
                next_song, times = next_songs[index]

                if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                    songs = [song for song, song_times in next_songs if song_times == times]
                    for song in song_ids:
                        if len(predicted_songs) >= 5:
                            break
                        if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                            part21new1 += 1
                            predicted_songs.append(song)
                    for song in songs:
                        if len(predicted_songs) >= 5:
                            break
                        if (song not in predicted_songs) and (songs not in query_song):
                            part21new1 += 1
                            predicted_songs.append(song)
                    index += len(songs)
                    continue
                index += 1
                if next_song not in predicted_songs and (next_songs not in query_song):
                    part21new1 += 1
                    predicted_songs.append(next_song)
                if len(predicted_songs) >= 5:
                    break

            if len(predicted_songs) >= 5:
                break

    if 1 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            for j in range(len(predicted_songs)):
                if i==j:
                    continue
                start_song = (query_song[-2] + query_song[-1] + predicted_songs[i] + predicted_songs[j])
                next_songs = cfd_5grams.most_common(start_song)
    
                index = 0
                while index < len(next_songs) and len(predicted_songs) < 5:
                    next_song, times = next_songs[index]
    
                    if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                        songs = [song for song, song_times in next_songs if song_times == times]
                        for song in song_ids:
                            if len(predicted_songs) >= 5:
                                break
                            if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                                part21new1new2 += 1
                                predicted_songs.append(song)
                        for song in songs:
                            if len(predicted_songs) >= 5:
                                break
                            if (song not in predicted_songs) and (songs not in query_song):
                                part21new1new2 += 1
                                predicted_songs.append(song)
                        index += len(songs)
                        continue
                    index += 1
    
                    if next_song not in predicted_songs and (next_songs not in query_song):
                        part21new1new2 += 1
                        predicted_songs.append(next_song)
                    if len(predicted_songs) >= 5:
                        break
    
                if len(predicted_songs) >= 5:
                    break

    if 1 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            for j in range(len(predicted_songs)):
                if i==j:
                    continue
                start_song = (query_song[-1] + predicted_songs[i] + predicted_songs[j])
                next_songs = cfd_4grams.most_common(start_song)
    
                index = 0
                while index < len(next_songs) and len(predicted_songs) < 5:
                    next_song, times = next_songs[index]
    
                    if index + 1 < len(next_songs) and times == next_songs[index + 1][1]:
                        songs = [song for song, song_times in next_songs if song_times == times]
                        for song in song_ids:
                            if len(predicted_songs) >= 5:
                                break
                            if (song in songs) and (song not in predicted_songs) and (songs not in query_song):
                                part1new1new2 += 1
                                predicted_songs.append(song)
                        for song in songs:
                            if len(predicted_songs) >= 5:
                                break
                            if (song not in predicted_songs) and (songs not in query_song):
                                part1new1new2 += 1
                                predicted_songs.append(song)
                        index += len(songs)
                        continue
                    index += 1
    
                    if next_song not in predicted_songs and (next_songs not in query_song):
                        part1new1new2 += 1
                        predicted_songs.append(next_song)
                    if len(predicted_songs) >= 5:
                        break
    
                if len(predicted_songs) >= 5:
                    break

    '''
    pyserini 區域
    預計做 MLE and BM25 的混合模型
    '''
    if len(predicted_songs) < 5:
        for song in song_ids:
            if user_code == 2 and album != song_to_album.get(song, ' '):
                continue
            if user_code == 1 and artist != song_to_artist.get(song, ' '):
                continue
            if (song not in query_song) and (song not in predicted_songs):
                predicted_songs.append(song)
                part_mle += 1
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        for song in song_ids:
            if (song not in query_song) and (song not in predicted_songs):
                predicted_songs.append(song)
                part_mle += 1
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        predicted_songs.append('18a62aea3e0e67e21ea56c125c29c474')
    
    if len(predicted_songs) < 5:
        predicted_songs.append('85422f927d88358292985cb319d216fa')
    
    if len(predicted_songs) < 5:
        predicted_songs.append('ef71212934e35b400232a0cd8e7e67a2')
        
    if len(predicted_songs) < 5:
        predicted_songs.append('4a27803802d6090389507671d8aed6eb')
        
    if len(predicted_songs) < 5:
        predicted_songs.append('bc58d186329eda8b23e510ba98b2c1ea')
        
    # print("Predicted Songs:", predicted_songs)
    submit_rows.append([key, *predicted_songs])

100%|██████████| 143064/143064 [00:05<00:00, 26798.57it/s]


In [18]:
# 重複的歌曲
print(part_repeated)
# 第一層（正統 n gram)
print(part4321, part321, part21)
# 第二層（擴展 n gram 範圍）
print(part5432, part432, part6543, part543, part32)
# 第三層（拿預測預測）
print(part321new1, part21new1, part21new1new2, part1new1new2, part_mle)

8327
144615 26481 126802
86744 3417 80830 2351 32772
36 64 24 694 201981


In [19]:
columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
output_df = pd.DataFrame(submit_rows, columns=columns)
output_df.to_csv('ngrams_jmlm0.9999_token10.csv', index=False)

In [14]:
output_df

,session_id,top1,top2,top3,top4,top5
0,8,eab84a27a694036cdf0b1b0affaf6bf3,76723b980445c7c7b1350ca038348bbb,0d97841ac6d6c5b03f30074449e4c879,254700cdaf28bff608018648d4bcf4ed,0cdde4abdac3c1f276790d456ca2e522
1,9,7653de936f26c9b71c728e88cdd29c1a,d1783056f15730a5ca9da967d9330a90,caffbd827ca36cc820bb22455f801dbb,c3827b4368f57b71579ce816080793a7,a8f678c4bbc7874cecf44478a7598a14
2,18,9cb10772a82000f45de4a950883df945,bb1e65ce2a87d4031bbdd51bf6123163,afae7aba2019b32d09bf3574329a07e6,1424a89b1f3edbfb475ddc907643fa4c,5f04347620b2bd6c364fd58eccc679e9
3,19,d6813a67f316942de54f967fda79abb2,f875c6647fcb2c445d7454c5fa9a3fc1,fdda2e66f17eb3a93cf8619b6cf8bb1f,3729d3bfa20571bd2ccf4134989d4a18,725e245060e4020289f6605db2d77bc3
4,28,20171b381e293bda502851778947ce57,95bf93c6eab031c297effa69bc9324ed,12092de14b33bc021a380f736bbfd4ae,049ec9a2140a73692863a84f5ee3ccc5,07b0c36755d8795849741a00c72539e4
...,...,...,...,...,...,...
143059,715299,e2ff3605e7ef9864e57496db028bdcff,16feb0de2b117983d507297be2782969,7eebcd3252bdf331b25b64f1a0f7b006,a2a05a34b38243df2aaa4996ee005f9d,787ca4169188e966f3dfc08fe0c549df
143060,715308,1b23dec79509ed41a743ac7336998db5,c24fcdfcdc1fe2b13c5c33af5202ef01,7012ff198297aaa9640d007f839d996f,f5c45775471ef8f30808be2c835f55da,5cd338e484fd1caf9970bdd0ad270241
143061,715309,b83d7912b15c97dea1ec15d9ef7acf6e,d447994544fb6e0f56dccb8d3b36fcf1,09aa4f19a7299905f8151ddb3c885755,5b5a586f8f410142c7bae2b9d774fcd1,00e5eafb6adf6ba6d606d93c6839ad00
143062,715318,3752912c9a7012dbd8b7f1cb75cebeb9,50365e0974d0530b98daf8e14d4102af,9948578a138f7d96c80ec50f26d1c57c,ba89ed3637491a12117db6a68dfbd979,8b75df62f58ccaabc7792c8b03180ebc


In [15]:
'''
150299 27066 128429
87175 3417 80830 2351 32772
3 6 1 633 202159
'''

'\n150299 27066 128429\n87175 3417 80830 2351 32772\n3 6 1 633 202159\n'

In [16]:
output_df.isnull().sum()

session_id    0
top1          0
top2          0
top3          0
top4          0
top5          0
dtype: int64